<img src="https://www3.um.edu.uy/logoum.jpg" width=300>
<h1 align="center">Thesis - KNN Item Item RMSE</h1> 
<h2 align="center">Alejo Paullier</h2> 

https://lkpy.lenskit.org/en/stable/knn.html

In [1]:
from lenskit import batch, topn, util
from lenskit import crossfold as xf
from lenskit.algorithms import Recommender, item_knn as knn
from lenskit.metrics.predict import rmse
from lenskit import topn
import pandas as pd
import numpy as np


ratings = pd.read_csv('C:\\Users\\Alejo\\Tesis\\Demo\\ml-100k\\u.data', sep='\t',
                      names=['user', 'item', 'rating', 'timestamp'])


def eval(aname, algo, train, test):
    fittable = util.clone(algo) # Object cloning means to create an exact copy of the original object.
    fittable = Recommender.adapt(fittable) 
    fittable.fit(train) # Entrenamos el algoritmo con el training dataset
    users = test.user.unique() # Devuelve un array con los users unicos en el testing dataset
    # now we run the recommender
    recs = batch.recommend(fittable, users, 100)
    # add the algorithm name for analyzability
    recs['Algorithm'] = aname
    return recs

# Define algorithm

In [8]:
algo_ii = knn.ItemItem(2,aggregate='weighted-average')

# Train algorithm with some data

In [9]:
algo_ii.fit(ratings)

# Generate some predictions

In [13]:
preds = batch.predict(algo_ii, ratings)
preds.head()

# La columna "prediction" es una estimación de que rating le asignaría el usuario X al item Y dado el algoritmo
# que se le pasa a "batch.predict"

,user,item,rating,timestamp,prediction
0,196,242,3,881250949,3.468619
1,186,302,3,891717742,4.386858
2,22,377,1,878887116,0.437298
3,244,51,2,880606923,2.658149
4,166,346,1,886397596,3.624622


# Compute per-user RMSE 

In [31]:
user_rmse = preds.groupby('user').apply(lambda df: rmse(df.prediction, df.rating))
print(user_rmse.mean())
user_rmse.head()

0.8756399790479684


user
1    0.782914
2    0.891713
3    0.981827
4    0.953156
5    0.954410
dtype: float64

# Global RMSE

In [32]:
rmse(preds['prediction'], preds['rating'])

0.8410757906739699

# Example

In [30]:
fr = pd.DataFrame([[2,8],[4,5]])
rmse(fr[0],fr[1])

### Example: RMSE = sqrt( [(2-8)^2 + (4-5)^2]/2  ) = 4.301162633521313

4.301162633521313